# Linear regression


The goal of this notebook is get you familiar with the ideas behind linear regression and how to implement this method in python. 

### What is linear regression?

How well can you fit a line to data points by eye? https://fditraglia.shinyapps.io/regression/

When we have a point cloud, or here measurements of two variables, we might notice the variables seem to lie near a line. Linear regression helps us find the line that describes our points the 'best'.

Today we'll use [statsmodels](https://www.statsmodels.org/stable/index.html) to perform our linear regression. Another popular packages is [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html), which we'll use in part 2.

Thanks to Sara Casella for providing the data and much of the code for this tutorial :)


In [1]:
# Import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std


## Data

Let's look at the presumed relationship between SAT scores and college gpa. Does scoring higher on the SAT really suggest the student will perform better in college? If I worked to gain 10 more points on the SAT, how much should I expect my college GPA to increase?

We'll read in a large data frame, but we will begin with focusing on the following variables:
 
**colgpa**:   College GPA on a four point scale

**sat**:      Combined SAT score (verbal plus math)

In [2]:
# Import data

collegeDataAll = pd.read_csv('data/college_gpa.csv')
collegeData = collegeDataAll[['sat','colgpa']]
collegeData.head()

# collegeData.describe()

FileNotFoundError: [Errno 2] File b'data/college_gpa.csv' does not exist: b'data/college_gpa.csv'

In [ ]:
# Plot

# We could also make numpy arrays from our pandas dataframe so that our plotting would look just like last week.
# When using a pandas dataframe with seaborn, we input the whole dataframe and then assign the x and y variables
# using the column names.
sns.scatterplot(data = collegeData, x='sat', y='colgpa', alpha = 0.2)  # Why is the alpha = 0.2 helpful? 

Yikes, looks like a big blob. Though perhaps there is some relationship between SAT score and gpa...

## Performing linear regression.

Let's hypothesize that our data should lie along a line. 

Recall the equation of a line is
\begin{equation}
y = a + bx
\end{equation}

where $a$ is the y-intercept and $b$ is the slope of the line.

Let's replace x and y with our variables. This is the line we are trying to find.

\begin{equation}
\text{college GPA} = a + b \text{SAT score}
\end{equation}

As we saw before, we could choose many values for $a$ or $b$ that result in a decent-fitting line. But which one is the _best_ fitting line? We say the line of best fit is the line that minimimzes the sum of squared vertical deviations, also called the sum of squared residuals.

In [ ]:
# Perform linear regression usings statsmodels

# The formula argument tells statsmodels what we think the linear relationship is. If we write down our equtaion (above)
# as y = a + bx, then we enter y ~ x into the function. The function will add the y-intercept term, a, and a coefficient,
# b, to any term on the right hand side.
model = smf.ols(formula = 'colgpa ~ sat', data = collegeData)
results = model.fit()
print(results.summary())

That table can be overwhelming. Let's extract just what we need.


In [ ]:
# Extract results

print('a=', results.params[0])
print('b=', results.params[1])

### YAY we did our first linear regression!

Pretty simple huh? Let's discuss what we found so far.

1. The slope $b$ is positive, which supports the idea that higher SAT scores correspond to higher GPAs.
2. 100 points more in the SAT are associated to $100*0.0019 = 0.19$ points more in the college GPA.

How good was our estimate of these parameters?

## Uncertainty

Just like when we estimated the mean of a population from a sample, above we have a sample and we are estimating the slope $b$ of the _real_ regression line (more on this later). With any estimate from a sample, we expect some error. Just like with the sample mean, we can calculate the _standard error of the slope_ which quantifies the uncertainty in our estimate. Low standard error indicates a more certain estimate. 

Similarly, we can also calcuate confidence intervals for our estimates of slope and y-intercept.



In [ ]:
# Extract results
print("Standard error of b=", results.bse[0])
print("Confidence interval for b=[0.002, 0.002]")
print("Confidence interval for a=[0.526, 0.800]")

Now that we have this linear description of the relationship, we can use the slope and intercept estimates to make predictions for _new_ data. 

**Prediction**: if your SAT score was 1300, your college GPA will be around to $0.663 + 1300*0.0019 = 3.13$ 

In [ ]:
print('Predicted values: ', results.predict())

sns.scatterplot(data = collegeData, x='sat', y='colgpa', alpha = 0.2, label = "data")
sns.lineplot(collegeData["sat"], results.predict(), color="navy", label = "Line of best fit")

In [ ]:
# To simplify our plotting, we can also use seaborn regplot

sns.regplot(data = collegeData, x='sat', y='colgpa', scatter_kws={'alpha':0.2})

## Multivariate linear regression

Let's do another linear regression, with two variables now. Let's add high school performance to the list of variables that might predict college GPA.
 
The variables we are interested in now are:
 
**colgpa**:   College GPA on a four point scale

**sat**:      Combined SAT score (verbal plus math)

**hsperc**   Percentile in HS graduating class (hsperc = 5 top 5%)

and our statistical model is 

\begin{equation}
\text{college GPA} = a + b_1 \ \text{SAT score} +  b_2 \ \text{HS percentile} 
\end{equation}


In [ ]:
# Let's first have a look at what hsperc looks like
sns.distplot(collegeDataAll['hsperc'])


In [ ]:
# Run the regression

model = smf.ols(formula = 'colgpa ~  sat + hsperc', data = collegeDataAll)
results = model.fit()
print(results.summary())

### Interpretation

- What is the coefficient on HS percentile?

- What happened to the coefficient on SAT score once we introduced HS percentile? Why do you think that is?

**Research** : how many variables and which variables should I include in a regression? This is not an easy question at all! :)


## Data 2: Standing long jumps

Do taller people jump further in a standing long jump in general? Let's examine the data we collected.

In [ ]:
## Data

heights = np.array([69. , 63. , 71. , 66. , 71. , 61. , 67. , 65. , 64. , 66. , 74. ,
       68. , 67. , 68. , 67. , 73. , 62. , 75. , 68. , 66. , 64. , 67. ,
       69. , 67. , 69. , 75. , 67. , 60. , 72. , 63. , 75.5, 68. , 66. ,
       61. , 70. , 62. , 62. , 63. , 65. , 68. , 63.5, 69. , 52. , 69. ,
       71.5, 73. , 70. , 65. , 64. , 64. ])
jump1 = np.array([ 50.2, 72, 84, 80, 56.5, 43, 78, 75, 39, 35, 87, 72, 92, 97, 72.5,
       100, 51.5, 85, 63, 62, 51, 33, 89, 79, 75.5, 80, 86, 66, 86, 58,
       102, 63, 44, 43, 92, 54.5, 48, 25, 43, 68, 54.5, 100, 88,
       100, 65, 87, 54.25, 41, 34, 71.3])
jump2 = np.array([57.4, 72.5, 78, 81, 60.5, 45, 75, 74, 40, 34, 90, 75, 94, 101, 69,
       105, 46, 91, 70, 60.5, 53, 35, 87, 78, 82, 83, 88, 75, 89, 59, 103,
       65, 62, 48, 95, 62, 40, 35, 43, 70, 54.5, 113, 84, 107,
       70, 90, 55.75, 37, 32, 74.6])
avg_jump = np.mean([jump1, jump2], axis=0)

# Let's put the data into a pandas dataframe to match what we did before
jump_df = pd.DataFrame({"height": heights, "jump1": jump1, "jump2": jump2, "avg_jump": avg_jump})
jump_df.head()

In [ ]:
# Draw a scatterplot of the data



In [ ]:
# Run a linear regression to test the relationship between average jump distance and height

model2 = smf.ols(formula = 'avg_jump ~ height', data = jump_df)
results2 = model2.fit()

sns.regplot(heights,avg_jump)
plt.xlabel("Height (in)")
plt.ylabel("Avg Jump (in)")
plt.title(f"Relation between height and jump distance\n y={results2.params[0]} + {results2.params[1]}x");

## The effect of outliers

Let's say now we also included Byron Jones' jump of 146.75 inches. Byron is 73 inches tall. How does that change our regression line?

In [ ]:
heights_byron = np.append(heights,73)
avg_jump_byron = np.append(avg_jump,146.73)
jump_df2 = pd.DataFrame({"height": heights_byron, "avg_jump": avg_jump_byron})

plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
sns.regplot(heights,avg_jump)
plt.xlabel("Height (in)")
plt.ylabel("Avg Jump (in)")
plt.title(f"Height v jump distance\n y={results2.params[0]} + {results2.params[1]}x")



# Redo linear regression
model3 = smf.ols(formula = 'avg_jump ~ height', data = jump_df2)
results3 = model3.fit()

plt.subplot(1,2,2)

sns.regplot(heights_byron,avg_jump_byron)
sns.scatterplot([73], [146.73], color ="red", label = "Byron Jones")
plt.xlabel("Height (in)")
plt.ylabel("Avg Jump (in)")
plt.title(f"Height v jump distance including Byron\n y={results3.params[0]} + {results3.params[1]}x")


plt.subplots_adjust(bottom=None, right=1, top=None, wspace=0.5, hspace = 0.4)

**Outliers** can greatly alter the estimates and even direction of your regression line. If you have outliers, first examine why they might be an outlier (is there some underlying biology that greatly changes their behavior?), and *consider* removing. Perhaps the most ethical option is to show the fit with the outlier, and then show the fit after removing the outlier.

--------

## Confidence bands and confidence in predictions

What is that blue shaded area around the regression line plotted by regplot that we have expertly ignored until now? This is the 95% confidence band. 

The **confidence band** measures the precision of the predicted mean Y for each value of X. 

In [ ]:
# The confidence band is shown in red in the following plot

sns.regplot(heights, avg_jump, line_kws={"color": "red"})
plt.xlabel("Height (in)")
plt.ylabel("Avg Jump (in)")
plt.title("Confidence bands");

Is this the same as predicting an individual new y-value? 

Let's say I make a standing long jump. How confident can you be in the prediction of *my jump distance*?

In [ ]:
drB_height = 67
drB_jump = 70

sns.regplot(heights, avg_jump)
sns.scatterplot([drB_height], [drB_jump], color = "orange", s=100)
plt.xlabel("Height (in)")
plt.ylabel("Avg Jump (in)");

The confidence band only describes precision in predicting the *mean* Y value for the given X value. Individuals have variation, so in order to capture the precision in estimating a new individual data point, we need the prediction intervals.

The **prediction interval** meaures the precision of the predicted single Y-values for each X. 


In [ ]:
# Calculate and plot the prediction intervals

a_height = results2.params[0]
b_height = results2.params[1]

# Calculate the predicted average jump values by hand. Usually predicted values mathematically are written with a 'hat'.
avg_jump_hats = a_height + b_height*heights

# Calculate the sum of squared residuals by hand
MS_resid_jump = (np.sum((avg_jump - avg_jump_hats)**2))/(heights.shape[0] -2)

# Calculate the standard error
SE_predict_heights = np.sqrt(MS_resid_jump * ((1+1/heights.shape[0]1+((heights-np.mean(heights))**2)/(np.sum((heights-np.mean(heights))**2)))))
t = stats.t.ppf(0.975, heights.shape[0]-2)

# Calculate the prediction intervals
prediction_upper = avg_jump_hats - t*SE_predict_heights
prediction_lower = avg_jump_hats + t*SE_predict_heights

sns.regplot(heights,avg_jump)
sns.lineplot(heights, prediction_upper, color = "gray")
sns.lineplot(heights, prediction_lower, color="gray", label="Prediction intervals")
sns.scatterplot([drB_height], [drB_jump], color = "orange", s=100)
plt.xlabel("Height (in)")
plt.ylabel("Avg Jump (in)");